# Supervised Finetuning
The following notebook describes the supervised Finetuning-process.

## Start Training
To start the training, we have to restart the textgeneration-webui, using the following commands:
    
    conda activate textgen
    cd /home/[...]/text-generation-webui/
    python server.py --share --model models/Pre-ant-model --load-in-8bit

Thereby, we load the Pre-ant-model (that is the new base model in the supervised finetuning-step) in 8bit.
    
After the webui started, the following steps have been executed within the webui:
1. Switch to tab "Training"
2. Give the LoRA-file a new name - in our case "Pre_ant_adapter"
3. Adapt the Hyperparameters for the training:
    -	Training-epochs: 20
    -	Learning-rate: 3e^-4
    -	LoRA Rank: 8
    -	LoRA Alpha: 16
    -	Batch Size: 128
    -	Micro Batch Size: 4
    -	Cutoff Length: 256
    -	LR Scheduler: linear
4. Go to the Tab "Formatted Dataset"
5. Adapt the Dataselection:
    - Data-Format: alpaca-format
    - Dataset: Structured_training_dataset
    - Evaluation Dataset: None
6. Click "Start LoRA Training"

After 3 hours the new LoRA-adapter was saved to the folder "text-generation-webui/loras/Ant_adapter".

### Create Ant-model
The new Ant-model is created by merging the newly trained LORA-adapter (Ant-adapter) with the corresponding base model (Pre-ant-model). Afterwards, it is pushed to the huggingace hub "Ant_model_developers/Ant_model"  Therefore, we executed the following code from this notebook:

#### Load Base-model, tokenizer, LoRA-adapter

In [ ]:
!pip install transformers
!pip install torch
!pip install huggingface_hub
!pip install sentencepiece

In [ ]:
from peft import PeftModel
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
model_path = "text-generation-webui/models/Pre-ant-model/"

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    load_in_8bit=False,
    torch_dtype=torch.float16,
    device_map="auto",
    offload_folder="offload",
    trust_remote_code=True
)
model.config.use_cache = False

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

#### Combine base model with LoRA-adapter

In [ ]:
peft_model = PeftModel.from_pretrained(model, 'text-generation-webui/loras/Ant-adapter/', is_trainable=True)

#### Merge base model with LoRA-adapter

In [ ]:
merged_model = peft_model.merge_and_unload()
merged_model.eval()

#### Save merged model to text-generation-webui

In [ ]:
merged_model.save_pretrained("text-generation-webui/models/Ant-model/", safe_serializetion=True)
tokenizer.save_pretrained("text-generation-webui/models/Ant-model/")

#### Push model to Huggingface

In [ ]:
from huggingface_hub import login
login("XXXX")

In [ ]:
from huggingface_hub import push_to_hub
merged_model.push_to_hub("Ant-model-developers/Ant_model")
tokenizer.push_to_hub("Ant-model-developers/Ant_model")